In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
# https://towardsdatascience.com/calculating-distance-between-two-geolocations-in-python-26ad3afe287b
import haversine as hs
# https://www.geeksforgeeks.org/read-multiple-csv-files-into-separate-dataframes-in-python/
import os

import warnings
warnings.simplefilter("ignore")

In [2]:
epa = pd.read_csv('../data/epa_monthly.csv')

In [ ]:
epa = epa[epa['PM2.5 Monthly Mean'] > 0]

In [3]:
epa.head()

,Date Local,Latitude,Longitude,PM2.5 Monthly Mean
0,2019-01-31,30.497478,-87.880258,6.825000
1,2019-01-31,34.214263,-79.767347,7.600000
2,2019-01-31,26.687606,-80.219619,5.428571
3,2019-01-31,34.843895,-82.414585,7.016667
4,2019-01-31,28.596389,-81.362500,6.721739


### Create columns with lat lon to 100th decimal place

In [4]:
epa['lat_2dec'] = epa['Latitude'].map(lambda x: f'{x:.2f}').values[0]
epa['lon_2dec'] = epa['Longitude'].map(lambda x: f'{x:.2f}').values[0]

## Filter epa for month, if needed, use each month as the source_df

In [5]:
epa_1 = epa[epa['Date Local'].str.contains('2019-01-')]

In [6]:
epa_2 = jan_epa = epa[epa['Date Local'].str.contains('2019-02-')]

In [7]:
epa_3 = jan_epa = epa[epa['Date Local'].str.contains('2019-03-')]

In [8]:
epa_4 = epa[epa['Date Local'].str.contains('2019-04-')]

In [9]:
epa_5 = epa[epa['Date Local'].str.contains('2019-05-')]

In [10]:
epa_6 = epa[epa['Date Local'].str.contains('2019-06-')]

In [11]:
epa_7 = epa[epa['Date Local'].str.contains('2019-07-')]

In [12]:
epa_8 = epa[epa['Date Local'].str.contains('2019-08-')]

In [13]:
epa_9 = epa[epa['Date Local'].str.contains('2019-09-')]

In [14]:
epa_10 = epa[epa['Date Local'].str.contains('2019-10-')]

In [15]:
epa_11 = epa[epa['Date Local'].str.contains('2019-11-')]

In [16]:
epa_12 = epa[epa['Date Local'].str.contains('2019-12-')]

# Function

## Extract New Feature Function
### For extracting feature values from one dataset, based on latitude/longitude within range of another dataset

In [17]:
def extract_feature(new_column_name, source_df, extract_df, extract_df_lat_name, 
                    extract_df_lon_name, extract_df_value_column_name, lat_range = .1, lon_range = .1):

    '''
    Takes the latitude and logitude from epa dataset and latitude and logitude from second dataset, 
    finds the most similar lat and long, by running a haversine function, then extracts the value and inputs into the epa
    dataset row with the closest lat/lon
    
    input:
        new_column_name: the name you want to give to the new epa column
        source_df: dataset to pull search lat/lon from and apply new values column (epa)
        extract_df: name of dataset to extract value from
        extract_df_lat_name: name of latitude column from extract_df
        extract_df_lon_name: name of longitude column from extract_df
        extract_df_value_column_name: name of the column where the values to extract are located
        lat_range = range to search for latitudes, default is .1 unless specified
        lon_range = range to search for latitudes, default is .1 unless specified
    
    output: 
        epa dataset with new column - assign function, with inputs, to variable to get new df
    '''
    
    # create empty list for extracted values 
    # each value is extracted per row of the source_df and compared with lat/lon's in range from the extract_df
    extracted_values_list = [] 
    
    # uncomment for testing
    # source_df = source_df[:6]
    try:
        for i in range(len(source_df)):
            #### search extract df for latitudes in range of epa latitudes and epa latitudes +- .1

            ## create range of lat and lon from source_df to search within

            # create lat range
            lat_search = float(source_df.iloc[i]['lat_2dec']) 
            lat_search_c_ceiling = lat_search + lat_range 
            lat_search_c_floor = lat_search - lat_range

            # create lon range
            lon_search = float(source_df.iloc[i]['lon_2dec']) 
            lon_search_c_ceiling = lon_search + lon_range
            lon_search_c_floor = lon_search - lon_range

            ## subset extraction_df by lat/lon floors and ceilings

            # subset extraction df by latitudes in range of epa latitude for i
            # https://www.geeksforgeeks.org/numpy-where-in-python/
            lat_extraction_df = extract_df.loc[np.where(
                ((extract_df[extract_df_lat_name] >= lat_search_c_floor) & 
                 (extract_df[extract_df_lat_name] <= lat_search_c_ceiling))
            )].reset_index(drop = True)

            # subset lat_extraction_df by longitudes in range of epa longitude for i
            lon_from_lat_extraction_df = lat_extraction_df.loc[np.where(
                ((lat_extraction_df[extract_df_lon_name] >= lon_search_c_floor) & 
                 (lat_extraction_df[extract_df_lon_name] <= lon_search_c_ceiling))
            )]

            #### pull the row with the smallest haversine distance from the epa lat/lon

            # grab lat/lon of current row of source_df (i)
            source_df_i_lat = float(source_df.iloc[i]['Latitude'])
            source_df_i_lon = float(source_df.iloc[i]['Longitude'])

            # create list of haversine values in range of source_df lat/lon current row
            hav_list = []
            for j in range(len(lon_from_lat_extraction_df)):
                loc1 = (source_df_i_lat, source_df_i_lon)
                loc2 = (float(lon_from_lat_extraction_df.iloc[j][extract_df_lat_name]), float(lon_from_lat_extraction_df.iloc[j][extract_df_lon_name]))
                hav = hs.haversine(loc1, loc2)
                hav_list.append(float(hav))

            # add haversine column and values to current row's temporary subset dataframe
            lon_from_lat_extraction_df['haversine'] = hav_list

            # find extracted value from temporary subset dataframe based on minimum haversine
            hav_min = lon_from_lat_extraction_df['haversine'].min()
            extracted_row = lon_from_lat_extraction_df[lon_from_lat_extraction_df['haversine'] == hav_min].index
            # https://stackoverflow.com/questions/62235344/how-to-return-a-cell-value-from-pandas-instead-of-a-series
            extracted_value = lon_from_lat_extraction_df.loc[extracted_row, extract_df_value_column_name].values[0]
            extracted_values_list.append(extracted_value)

    except IndexError:
        print(f'Index Error at row {i}')

    # set new epa column with name
    source_df[new_column_name] = extracted_values_list
    
    return source_df

## Load CSV Function
### To extract each group of monthly files per feature folder

In [18]:
def get_monthly_datasets(file_path):
    '''
    Goes into a parent folder and creates a dataset for each csv file in that folder. Must have 12 files.

    file_path: file path for the parent folder of the files

    Output: A dataset for each file (assign the function to the dataset names, in sequence of files)
    '''
    # https://www.geeksforgeeks.org/read-multiple-csv-files-into-separate-dataframes-in-python/
    
    # assign path
    path, dirs, files = next(os.walk(file_path))
    file_count = len(files)
    # create empty list
    dataframes_list = []

    # append datasets to the list
    for i in range(file_count):
        if files[i].__contains__('Icon'):
            pass
        else:
            temp_df = pd.read_csv(file_path+files[i])
            print(file_path+files[i])
            dataframes_list.append(temp_df)
        
    return dataframes_list

# NO2

## Get AOD Files and Assign to New Datasets

In [19]:
no2_df_list = get_monthly_datasets('../data/NO2data_cleaned/2019/')

../data/NO2data_cleaned/2019/NO2_10.csv
../data/NO2data_cleaned/2019/NO2_04.csv
../data/NO2data_cleaned/2019/NO2_05.csv
../data/NO2data_cleaned/2019/NO2_11.csv
../data/NO2data_cleaned/2019/NO2_07.csv
../data/NO2data_cleaned/2019/NO2_12.csv
../data/NO2data_cleaned/2019/NO2_06.csv
../data/NO2data_cleaned/2019/NO2_02.csv
../data/NO2data_cleaned/2019/NO2_03.csv
../data/NO2data_cleaned/2019/NO2_01.csv
../data/NO2data_cleaned/2019/NO2_08.csv
../data/NO2data_cleaned/2019/NO2_09.csv


In [20]:
# assigned in order of get_monthly_datasets output
no2_10, no2_4, no2_5, no2_11, no2_7, no2_12, no2_6, no2_2, no2_3, no2_1, no2_8, no2_9 = [pd.DataFrame(each) for each in no2_df_list]

## Get NO2 Feature and Combine with Monthly EPA

In [21]:
epa_1 = extract_feature('no2', epa_1, no2_1, 'lat', 'lon', 'NO2', .5 , .5)

In [22]:
epa_2 = extract_feature('no2', epa_2, no2_2, 'lat', 'lon', 'NO2')

In [23]:
epa_3 = extract_feature('no2', epa_3, no2_3, 'lat', 'lon', 'NO2')

In [24]:
epa_4 = extract_feature('no2', epa_4, no2_4, 'lat', 'lon', 'NO2')

In [25]:
epa_5 = extract_feature('no2', epa_5, no2_5, 'lat', 'lon', 'NO2')

In [26]:
epa_6 = extract_feature('no2', epa_6, no2_6, 'lat', 'lon', 'NO2')

In [27]:
epa_7 = extract_feature('no2', epa_7, no2_7, 'lat', 'lon', 'NO2')

In [28]:
epa_8 = extract_feature('no2', epa_8, no2_8, 'lat', 'lon', 'NO2')

In [29]:
epa_9 = extract_feature('no2', epa_9, no2_9, 'lat', 'lon', 'NO2')

In [30]:
epa_10 = extract_feature('no2', epa_10, no2_10, 'lat', 'lon', 'NO2')

In [31]:
epa_11 = extract_feature('no2', epa_11, no2_11, 'lat', 'lon', 'NO2')

In [32]:
epa_12 = extract_feature('no2', epa_12, no2_12, 'lat', 'lon', 'NO2')

# AOD

## Get AOD Files and Assign to New Datasets

In [33]:
aod_df_list = get_monthly_datasets('../data/AODdata_cleaned/2019/')

../data/AODdata_cleaned/2019/AOD_08.csv
../data/AODdata_cleaned/2019/AOD_09.csv
../data/AODdata_cleaned/2019/AOD_01.csv
../data/AODdata_cleaned/2019/AOD_02.csv
../data/AODdata_cleaned/2019/AOD_03.csv
../data/AODdata_cleaned/2019/AOD_07.csv
../data/AODdata_cleaned/2019/AOD_12.csv
../data/AODdata_cleaned/2019/AOD_06.csv
../data/AODdata_cleaned/2019/AOD_10.csv
../data/AODdata_cleaned/2019/AOD_04.csv
../data/AODdata_cleaned/2019/AOD_05.csv
../data/AODdata_cleaned/2019/AOD_11.csv


In [34]:
# assigned in order of get_monthly_datasets output
aod_8, aod_9, aod_1, aod_2, aod_3, aod_7, aod_12, aod_6, aod_10, aod_4, aod_5, aod_11 = [pd.DataFrame(each) for each in aod_df_list]

## Get AOD Feature and Combine with Monthly EPA

In [35]:
epa_1 = extract_feature('aod', epa_1, aod_1, 'Latitude', 'Longitude', 'AOD_550_AVG')

In [36]:
epa_2 = extract_feature('aod', epa_2, aod_2, 'Latitude', 'Longitude', 'AOD_550_AVG')

In [37]:
epa_3 = extract_feature('aod', epa_3, aod_3, 'Latitude', 'Longitude', 'AOD_550_AVG')

In [38]:
epa_4 = extract_feature('aod', epa_4, aod_4, 'Latitude', 'Longitude', 'AOD_550_AVG')

In [39]:
epa_5 = extract_feature('aod', epa_5, aod_5, 'Latitude', 'Longitude', 'AOD_550_AVG')

In [40]:
epa_6 = extract_feature('aod', epa_6, aod_6, 'Latitude', 'Longitude', 'AOD_550_AVG', .5, .5)

In [41]:
epa_7 = extract_feature('aod', epa_7, aod_7, 'Latitude', 'Longitude', 'AOD_550_AVG')

In [42]:
epa_8 = extract_feature('aod', epa_8, aod_8, 'Latitude', 'Longitude', 'AOD_550_AVG')

In [43]:
epa_9 = extract_feature('aod', epa_9, aod_9, 'Latitude', 'Longitude', 'AOD_550_AVG')

In [44]:
epa_10 = extract_feature('aod', epa_10, aod_10, 'Latitude', 'Longitude', 'AOD_550_AVG')

In [45]:
epa_11 = extract_feature('aod', epa_11, aod_11, 'Latitude', 'Longitude', 'AOD_550_AVG')

In [46]:
epa_12 = extract_feature('aod', epa_12, aod_12, 'Latitude', 'Longitude', 'AOD_550_AVG')

# Temperature

## Get Temp Files and Assign to New Datasets

In [47]:
temp_df_list = get_monthly_datasets('../data/tempdata_cleaned/2019/')

../data/tempdata_cleaned/2019/temp_09.csv
../data/tempdata_cleaned/2019/temp_08.csv
../data/tempdata_cleaned/2019/temp_01.csv
../data/tempdata_cleaned/2019/temp_03.csv
../data/tempdata_cleaned/2019/temp_02.csv
../data/tempdata_cleaned/2019/temp_12.csv
../data/tempdata_cleaned/2019/temp_06.csv
../data/tempdata_cleaned/2019/temp_07.csv
../data/tempdata_cleaned/2019/temp_05.csv
../data/tempdata_cleaned/2019/temp_11.csv
../data/tempdata_cleaned/2019/temp_10.csv
../data/tempdata_cleaned/2019/temp_04.csv


In [48]:
temp_9, temp_8, temp_1, temp_3, temp_2, temp_12, temp_6, temp_7, temp_5, temp_11, temp_10, temp_4 = [pd.DataFrame(each) for each in temp_df_list]

## Extract Temp Feature and Combine with Monthly EPAs

In [49]:
epa_1 = extract_feature('temp', epa_1, temp_1, 'lat', 'lon', 'temp')

In [50]:
epa_2 = extract_feature('temp', epa_2, temp_2, 'lat', 'lon', 'temp')

In [51]:
epa_3 = extract_feature('temp', epa_3, temp_3, 'lat', 'lon', 'temp')

In [52]:
epa_4 = extract_feature('temp', epa_4, temp_4, 'lat', 'lon', 'temp')

In [53]:
epa_5 = extract_feature('temp', epa_5, temp_5, 'lat', 'lon', 'temp')

In [54]:
epa_6 = extract_feature('temp', epa_6, temp_6, 'lat', 'lon', 'temp')

In [55]:
epa_7 = extract_feature('temp', epa_7, temp_7, 'lat', 'lon', 'temp')

In [56]:
epa_8 = extract_feature('temp', epa_8, temp_8, 'lat', 'lon', 'temp')

In [57]:
epa_9 = extract_feature('temp', epa_9, temp_9, 'lat', 'lon', 'temp')

In [58]:
epa_10 = extract_feature('temp', epa_10, temp_10, 'lat', 'lon', 'temp')

In [59]:
epa_11 = extract_feature('temp', epa_11, temp_11, 'lat', 'lon', 'temp')

In [60]:
epa_12 = extract_feature('temp', epa_12, temp_12, 'lat', 'lon', 'temp')

# Precipitation

## Get Precip Files and Assign to New Datasets

In [61]:
precip_df_list = get_monthly_datasets('../data/precipdata_cleaned/2019/')

../data/precipdata_cleaned/2019/precip_02.csv
../data/precipdata_cleaned/2019/precip_03.csv
../data/precipdata_cleaned/2019/precip_01.csv
../data/precipdata_cleaned/2019/precip_04.csv
../data/precipdata_cleaned/2019/precip_10.csv
../data/precipdata_cleaned/2019/precip_11.csv
../data/precipdata_cleaned/2019/precip_05.csv
../data/precipdata_cleaned/2019/precip_07.csv
../data/precipdata_cleaned/2019/precip_06.csv
../data/precipdata_cleaned/2019/precip_12.csv
../data/precipdata_cleaned/2019/precip_08.csv
../data/precipdata_cleaned/2019/precip_09.csv


In [62]:
precip_2, precip_3, precip_1, precip_4, precip_10, precip_11, precip_5, precip_7, precip_6, precip_12, precip_8, precip_9 = [pd.DataFrame(each) for each in precip_df_list]

## Extract Precip Feature and Combine with Monthly EPAs

In [63]:
epa_1 = extract_feature('precip', epa_1, precip_1, 'lat', 'lon', 'precip')

In [64]:
epa_2 = extract_feature('precip', epa_2, precip_2, 'lat', 'lon', 'precip')

In [65]:
epa_3 = extract_feature('precip', epa_3, precip_3, 'lat', 'lon', 'precip')

In [66]:
epa_4 = extract_feature('precip', epa_4, precip_4, 'lat', 'lon', 'precip')

In [67]:
epa_5 = extract_feature('precip', epa_5, precip_5, 'lat', 'lon', 'precip')

In [68]:
epa_6 = extract_feature('precip', epa_6, precip_6, 'lat', 'lon', 'precip')

In [69]:
epa_7 = extract_feature('precip', epa_7, precip_7, 'lat', 'lon', 'precip')

In [70]:
epa_8 = extract_feature('precip', epa_8, precip_8, 'lat', 'lon', 'precip')

In [71]:
epa_9 = extract_feature('precip', epa_9, precip_9, 'lat', 'lon', 'precip')

In [72]:
epa_10 = extract_feature('precip', epa_10, precip_10, 'lat', 'lon', 'precip')

In [73]:
epa_11 = extract_feature('precip', epa_11, precip_11, 'lat', 'lon', 'precip')

In [74]:
epa_12 = extract_feature('precip', epa_12, precip_12, 'lat', 'lon', 'precip')

# Concatenate EPA monthly datasets!

In [75]:
epa_monthly_list = [epa_1, epa_2, epa_3, epa_4, epa_5, epa_6, epa_7, epa_8, epa_9, epa_10, epa_11, epa_12]

In [76]:
epa_final = pd.DataFrame()

for each in epa_monthly_list:
    epa_final = pd.concat([epa_final, each])

In [77]:
epa_final.sort_values('Date Local', inplace = True)

In [78]:
epa_final.dropna(inplace = True)

In [79]:
epa.head()

,Date Local,Latitude,Longitude,PM2.5 Monthly Mean,lat_2dec,lon_2dec
0,2019-01-31,30.497478,-87.880258,6.825000,30.50,-87.88
1,2019-01-31,34.214263,-79.767347,7.600000,30.50,-87.88
2,2019-01-31,26.687606,-80.219619,5.428571,30.50,-87.88
3,2019-01-31,34.843895,-82.414585,7.016667,30.50,-87.88
4,2019-01-31,28.596389,-81.362500,6.721739,30.50,-87.88


In [80]:
epa_final.head()

,Date Local,Latitude,Longitude,PM2.5 Monthly Mean,lat_2dec,lon_2dec,no2,aod,temp,precip
0,2019-01-31,30.497478,-87.880258,6.825000,30.50,-87.88,1.541665e+15,0.052667,290.88,3.993249
780,2019-01-31,38.013333,-87.577222,9.860000,30.50,-87.88,1.120693e+15,0.035250,290.56,4.091899
779,2019-01-31,41.874999,-90.177574,10.290000,30.50,-87.88,1.374970e+15,0.201500,288.74,4.167067
778,2019-01-31,38.013333,-87.577222,7.222581,30.50,-87.88,1.120693e+15,0.035250,290.56,4.091899
777,2019-01-31,41.164400,-81.235200,8.762500,30.50,-87.88,1.421505e+15,0.035250,290.56,4.091899


In [89]:
# epa_final.to_csv('../data/epa_final.csv', index = False)

In [82]:
# epa_final['Date Local'] = pd.to_datetime(epa_final['Date Local'])

In [83]:
# epa_final.set_index('Date Local', inplace = True)

In [84]:
# plot = epa_final[['PM2.5 Monthly Mean', 'aod', 'temperature', 'precipitation']].dropna()
# plot = plot.resample('M').mean()
# plot.head(12)

In [85]:
# plt.figure(figsize = (10, 6))
# plt.plot(plot['PM2.5 Monthly Mean'])
# plt.ylabel('Average Monthly PM2.5');

In [86]:
# plt.figure(figsize = (10, 6))
# plt.plot(plot['aod'])
# plt.ylabel('Average Monthly AOD');

In [87]:
# plt.figure(figsize = (10, 6))
# plt.plot(plot['temperature'])
# plt.ylabel('Average Monthly Temperature');

In [88]:
# plt.figure(figsize = (10, 6))
# plt.plot(plot['precipitation'])
# plt.ylabel('Average Monthly Precipitation');